In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
import statsmodels.api as sm
import warnings

from optimisation_markowitz import MarkowitzOptimization
from optimisation_simm import SingleFactorModel, SingleFactorModelOptimization
from utils import Utils

warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2

In [2]:
start_date = "2023-04-01"
end_date = "2024-04-01"

We will use S&P500 Information Technology Index as benchmark to evaluate the performance with Single Index Market Model and Constant Correlation Model. The return over four years will be used as the market return in the formula

In [6]:
# Get close data for S&P 500 Information Technology Index
market_index_data = Utils.get_historical_data("^SP500-45", start_date, end_date)

# calculate simple return for market index
market_index_simple_return = Utils.calculate_simple_return(market_index_data)

# calculate daily market data returns.
# use that to calculate expected returns and variance
market_index_daily_returns = Utils.calculate_daily_returns(market_index_data)
market_index_data_expected_return = market_index_daily_returns.mean()
market_index_data_variance = market_index_daily_returns.var()

print(f"Market Index Simple Return from {start_date} - {end_date} in %: ", market_index_simple_return * 100)
print("Market Index Expected daily Return in %: ", market_index_data_expected_return * 100)
print("Market Index daily Variance in %: ", market_index_data_variance * 100)

[*********************100%%**********************]  1 of 1 completed
Market Index Simple Return from 2023-04-01 - 2024-04-01 in %:  44.858555542505044
Market Index Expected daily Return in %:  0.1563335685653037
Market Index daily Variance in %:  0.013689625923552962


For markowitz model, we will use 3-Month US Treasury Bill as risk-free rate. Since we are holding the portfolio for 1 year, we will take the last value as the risk-free rate.

In [7]:
risk_free_rate_df = yf.download("^IRX", start="2023-04-01", end="2024-04-01")
risk_free_rate = risk_free_rate_df['Adj Close'].iloc[-1] / 100
risk_free_rate

[*********************100%%**********************]  1 of 1 completed


0.05203000068664551

We will now download the data for US Technology Equity ETFs on which we will run our strategies. We will use the following top 5 ETFs based on total assets:
1. Vanguard Information Technology ETF (VGT)
2. Technology Select Sector SPDR Fund (XLK)
3. VanEck Semiconductor ETF (SMH)
4. iShares US Technology ETF (IYW)
5. iShares Semiconductor ETF (SOXX)

Source: https://etfdb.com/etfdb-category/technology-equities/ 

In [8]:
# get the data for VanGuard Information Technology ETF
vgt_data = Utils.get_historical_data("VGT", start_date, end_date)

[*********************100%%**********************]  1 of 1 completed


In [9]:
# get the data for Technology Select Sector SPDR Fund
xlk_data = Utils.get_historical_data("XLK", start_date, end_date)

[*********************100%%**********************]  1 of 1 completed


In [10]:
# get the data for VanEck Semiconductor ETF
smh_data = Utils.get_historical_data("SMH", start_date, end_date)

[*********************100%%**********************]  1 of 1 completed


In [11]:
# get the data for iShares US Technology ETF
iyt_data = Utils.get_historical_data("IYW", start_date, end_date)

[*********************100%%**********************]  1 of 1 completed


In [12]:
# get the data for iShares Semiconductor ETF
soxx_data = Utils.get_historical_data("SOXX", start_date, end_date)

[*********************100%%**********************]  1 of 1 completed


In [13]:
# check for any missing data
print(market_index_data.isnull().sum())
print(vgt_data.isnull().sum())
print(xlk_data.isnull().sum())
print(smh_data.isnull().sum())
print(iyt_data.isnull().sum())
print(soxx_data.isnull().sum())

Adj Close        0
simple_return    0
dtype: int64
Adj Close    0
dtype: int64
Adj Close    0
dtype: int64
Adj Close    0
dtype: int64
Adj Close    0
dtype: int64
Adj Close    0
dtype: int64


In [16]:

vgt_simple_return = Utils.calculate_simple_return(vgt_data)
xlk_simple_return = Utils.calculate_simple_return(xlk_data)
smh_simple_return = Utils.calculate_simple_return(smh_data)
iyt_simple_return = Utils.calculate_simple_return(iyt_data)
soxx_simple_return = Utils.calculate_simple_return(soxx_data)

simple_returns_dict = {
    "VGT": vgt_simple_return,
    "XLK": xlk_simple_return,
    "SMH": smh_simple_return,
    "IYT": iyt_simple_return,
    "SOXX": soxx_simple_return
}

for ticker, simple_return in simple_returns_dict.items():
    print(f"{ticker} -> Simple Return from {start_date} - {end_date} in %: ", simple_return * 100)

VGT -> Simple Return from 2023-04-01 - 2024-04-01 in %:  37.26375002326796
XLK -> Simple Return from 2023-04-01 - 2024-04-01 in %:  39.25822244042367
SMH -> Simple Return from 2023-04-01 - 2024-04-01 in %:  72.98801691288277
IYT -> Simple Return from 2023-04-01 - 2024-04-01 in %:  46.16351592656645
SOXX -> Simple Return from 2023-04-01 - 2024-04-01 in %:  57.11975942229284


In [17]:
# find the variance-covriance matrix of the returns
# create a dataframe with the returns of all the ETFs
# calculate the variance-covariance matrix
vga_daily_returns = Utils.calculate_daily_returns(vgt_data)
xlk_daily_returns = Utils.calculate_daily_returns(xlk_data)
smh_daily_returns = Utils.calculate_daily_returns(smh_data)
iyt_daily_returns = Utils.calculate_daily_returns(iyt_data)
soxx_daily_returns = Utils.calculate_daily_returns(soxx_data)


daily_returns_df = pd.DataFrame([vga_daily_returns, xlk_daily_returns, smh_daily_returns, iyt_daily_returns, soxx_daily_returns]).T 
daily_returns_df.columns = ["VGT", "XLK", "SMH", "IYT", "SOXX"]

cov_matrix = daily_returns_df.cov()
cov_matrix

,VGT,XLK,SMH,IYT,SOXX
VGT,0.000125,0.000122,0.000171,0.000127,0.000169
XLK,0.000122,0.000122,0.000167,0.000125,0.000164
SMH,0.000171,0.000167,0.000298,0.000173,0.000294
IYT,0.000127,0.000125,0.000173,0.000133,0.000169
SOXX,0.000169,0.000164,0.000294,0.000169,0.000304


In [79]:
# standard deviations between daily returns
std_dev = daily_returns_df.std()
std_dev

VGT     0.011188
XLK     0.011044
SMH     0.017272
IYT     0.011536
SOXX    0.017427
dtype: float64

## Optimal tangency portfolio using Markowitz model 

In [18]:
# Run the Markowitz Portfolio Optimization
simple_returns_list = list(simple_returns_dict.values())
markowitz = MarkowitzOptimization(simple_returns_list, cov_matrix, risk_free_rate)


In [22]:
# find the weights of the portfolio with short shelling
markowitz_weights_with_ss = markowitz.find_tangency_portfolio_with_short_selling()

In [26]:
# create the wieghts dictionary
markowitz_weights_with_ss_dict = {
    "VGT": markowitz_weights_with_ss[0],
    "XLK": markowitz_weights_with_ss[1],
    "SMH": markowitz_weights_with_ss[2],
    "IYT": markowitz_weights_with_ss[3],
    "SOXX": markowitz_weights_with_ss[4]
}

for ticker, weight in markowitz_weights_with_ss_dict.items():
    print(f"{ticker} -> Markowitz Weights with Short Selling in %: ", weight)

VGT -> Markowitz Weights with Short Selling in %:  -20.111840703383262
XLK -> Markowitz Weights with Short Selling in %:  1.695950695797694
SMH -> Markowitz Weights with Short Selling in %:  11.73309089630365
IYT -> Markowitz Weights with Short Selling in %:  15.755884608973975
SOXX -> Markowitz Weights with Short Selling in %:  -8.073085497692059


In [28]:
# find the weights of the portfolio without short shelling
markowitz_weights_without_ss = markowitz.find_tangency_portfolio_no_short_selling()

In [45]:
# weights dictionary
markowitz_weights_without_ss_dict = {
    "VGT": markowitz_weights_without_ss[0],
    "XLK": markowitz_weights_without_ss[1],
    "SMH": markowitz_weights_without_ss[2],
    "IYT": markowitz_weights_without_ss[3],
    "SOXX": markowitz_weights_without_ss[4]
}

for ticker, weight in markowitz_weights_without_ss_dict.items():
    print(f"{ticker} -> Markowitz Weights without Short Selling in %: ", weight * 100)

VGT -> Markowitz Weights without Short Selling in %:  8.599643094562447e-13
XLK -> Markowitz Weights without Short Selling in %:  0.0
SMH -> Markowitz Weights without Short Selling in %:  79.17754892463019
IYT -> Markowitz Weights without Short Selling in %:  20.82245107536425
SOXX -> Markowitz Weights without Short Selling in %:  6.1284092352496896e-12


In [35]:
expected_return_with_ss, volatility_with_ss = Utils.calculate_portfolio_return_and_risk(markowitz_weights_with_ss, simple_returns_list, cov_matrix)
print("Expected Return with Short Selling in %: ", expected_return_with_ss * 100)
print("Volatility with Short Selling in %: ", volatility_with_ss * 100)

Expected Return with Short Selling in %:  439.726770624501
Volatility with Short Selling in %:  6.690023441816276


In [36]:
# calculate sharpe ratio
sharpe_ratio_with_ss = Utils.calculate_sharpe_ratio(expected_return_with_ss, volatility_with_ss, risk_free_rate)
print("Sharpe Ratio with Short Selling: ", sharpe_ratio_with_ss)

Sharpe Ratio with Short Selling:  83.2282801206712


In [37]:
expected_return_without_ss, volatility_without_ss = Utils.calculate_portfolio_return_and_risk(markowitz_weights_without_ss, simple_returns_list, cov_matrix)
print("Expected Return without Short Selling in %: ", expected_return_without_ss * 100)
print("Volatility without Short Selling in %: ", volatility_without_ss * 100)

Expected Return without Short Selling in %:  67.4024983187962
Volatility without Short Selling in %:  1.5802974718769525


In [38]:
# calculate sharpe ratio
sharpe_ratio_without_ss = Utils.calculate_sharpe_ratio(expected_return_without_ss, volatility_without_ss, risk_free_rate)
print("Sharpe Ratio without Short Selling: ", sharpe_ratio_without_ss)

Sharpe Ratio without Short Selling:  12.650816832261501


From the results above, we observe that as expected, the returns after allowing short shelling are much higher than when short selling is not allowed. Furthermore, the sharpe ratio from allowing short selling is around 5 times larger than that when short selling is not allowed

For an investor not interested in short selling, this is still a good strategy as it exceeds market returns by more than 20%. Furthermore, if an investory is interested in diversifying the portflio more, we can put a constraint on the percentage invested in each asset. We will try this below

In [42]:
# we are interested in finding optimal weights when an asset can hold 30% of the portfolio max
markowitz_weights_without_ss_pct_constraint = markowitz.find_tangency_portfolio_no_short_selling_and_max_weight(0.3)

In [44]:
markowitz_weights_dict_without_ss_pct_constraint = {
    "VGT": markowitz_weights_without_ss_pct_constraint[0],
    "XLK": markowitz_weights_without_ss_pct_constraint[1],
    "SMH": markowitz_weights_without_ss_pct_constraint[2],
    "IYT": markowitz_weights_without_ss_pct_constraint[3],
    "SOXX": markowitz_weights_without_ss_pct_constraint[4]
}

for ticker, weight in markowitz_weights_dict_without_ss_pct_constraint.items():
    print(f"{ticker} -> Markowitz Weights without Short Selling and 30% constraint in %: ", weight * 100)

VGT -> Markowitz Weights without Short Selling and 30% constraint in %:  5.561197145804844
XLK -> Markowitz Weights without Short Selling and 30% constraint in %:  30.0000000000555
SMH -> Markowitz Weights without Short Selling and 30% constraint in %:  30.000000000186887
IYT -> Markowitz Weights without Short Selling and 30% constraint in %:  30.00000000007597
SOXX -> Markowitz Weights without Short Selling and 30% constraint in %:  4.438802853876809


In [49]:
expected_return_with_ss_with_constraint, volatility_with_ss_with_constraint = Utils.calculate_portfolio_return_and_risk(markowitz_weights_without_ss_pct_constraint, simple_returns_list, cov_matrix)
print("Expected Return with Short Selling and Constraint in %: ", expected_return_with_ss_with_constraint * 100)
print("Volatility with Short Selling and Constraint in %: ", volatility_with_ss_with_constraint * 100)

Expected Return with Short Selling and Constraint in %:  52.13067069823326
Volatility with Short Selling and Constraint in %:  1.2882928203635993


In [51]:
# find the Sharpe Ratio
sharpe_no_ss_with_constraint = Utils.calculate_sharpe_ratio(expected_return_with_ss_with_constraint, volatility_with_ss_with_constraint, risk_free_rate)
print("Sharpe Ratio with Short Selling and Constraint: ", sharpe_no_ss_with_constraint)

Sharpe Ratio with Short Selling and Constraint:  9.771742688237042


After adding a constraint that each stock can have maximum of 30% weight, the portfolio is diversified. The expected returns have decreased by 15% whereas portfolio risk have decrease by 0.3%

## Single Index Market Model

We will now compare results from markowitz model with single index market model. We will use S&P500 Information Technology Index as benchmark to evaluate the performance with Single Index Market Model.

From this point, we will run `SingleFactorModel` class over all ETFs calculating beta, alpha and tau. We will then calculate the expected return using the formula:

$$
u_i = \alpha_i + \beta_i \cdot u_m

In [53]:
def get_simm_params(asset_return: pd.DataFrame, market_return: pd.DataFrame):
  model = SingleFactorModel(asset_return, market_return)
  model.fit()
  
  params = model.get_params()
  alpha = params.get("alpha").get("value")
  beta = params.get("beta").get("value")
  tau = params.get("tau").get("value")
  
  return (alpha, beta, tau)
  

In [55]:
# run SIMM for VGT
vgt_alpha, vgt_beta, vgt_tau = get_simm_params(vga_daily_returns, market_index_daily_returns)

In [56]:
# run SIMM for XLK
xlk_alpha, xlk_beta, xlk_tau = get_simm_params(xlk_daily_returns, market_index_daily_returns)

In [57]:
# run SIMM for SMH
smh_alpha, smh_beta, smh_tau = get_simm_params(smh_daily_returns, market_index_daily_returns)

In [58]:
# run SIMM for IYT
iyt_alpha, iyt_beta, iyt_tau = get_simm_params(iyt_daily_returns, market_index_daily_returns)

In [59]:
# run SIMM for SOXX
soxx_alpha, soxx_beta, soxx_tau = get_simm_params(soxx_daily_returns, market_index_daily_returns)

In [60]:
# get expected returns using alpha, beta and expected market return
vgt_expected_return = vgt_alpha + (vgt_beta * market_index_data_expected_return)
xlk_expected_return = xlk_alpha + (xlk_beta * market_index_data_expected_return)
smh_expected_return = smh_alpha + (smh_beta * market_index_data_expected_return)
iyt_expected_return = iyt_alpha + (iyt_beta * market_index_data_expected_return)
soxx_expected_return = soxx_alpha + (soxx_beta * market_index_data_expected_return)

In [62]:
# we can now pack the expected returns into a dictionary along with beta and tau
expected_returns_dict = {
    "VGT": vgt_expected_return,
    "XLK": xlk_expected_return,
    "SMH": smh_expected_return,
    "IYT": iyt_expected_return,
    "SOXX": soxx_expected_return
}

alpha_dict = {
    "VGT": vgt_alpha,
    "XLK": xlk_alpha,
    "SMH": smh_alpha,
    "IYT": iyt_alpha,
    "SOXX": soxx_alpha
}

beta_dict = {
    "VGT": vgt_beta,
    "XLK": xlk_beta,
    "SMH": smh_beta,
    "IYT": iyt_beta,
    "SOXX": soxx_beta
}

tau_dict = {
    "VGT": vgt_tau,
    "XLK": xlk_tau,
    "SMH": smh_tau,
    "IYT": iyt_tau,
    "SOXX": soxx_tau
}

for i in range(len(expected_returns_dict.keys())):
  print(f"{ list(expected_returns_dict.keys())[i] } -> Expected Daily Return in %: ", list(expected_returns_dict.values())[i] * 100)
  print(f"{ list(alpha_dict.keys())[i] } -> Alpha: ", list(alpha_dict.values())[i])
  print(f"{ list(beta_dict.keys())[i] } -> Beta: ", list(beta_dict.values())[i])
  print(f"{ list(tau_dict.keys())[i] } -> Tau: ", list(tau_dict.values())[i])
  print()

VGT -> Expected Daily Return in %:  0.13402053852496243
VGT -> Alpha:  -0.00012983837755000072
VGT -> Beta:  0.9403250858343697
VGT -> Tau:  0.002036844094078624

XLK -> Expected Daily Return in %:  0.13968242938889514
XLK -> Alpha:  -6.718197476336831e-05
XLK -> Beta:  0.936463155090568
XLK -> Tau:  0.0013901279215019285

SMH -> Expected Daily Return in %:  0.2359472140425165
SMH -> Alpha:  0.0003236418568278154
SMH -> Beta:  1.3022348957299865
SMH -> Tau:  0.008150764625956512

IYT -> Expected Daily Return in %:  0.15978216261794406
IYT -> Alpha:  9.12318311877671e-05
IYT -> Beta:  0.963702043532858
IYT -> Tau:  0.0024403074884453745

SOXX -> Expected Daily Return in %:  0.19738981455821436
SOXX -> Alpha:  2.017572856687216e-05
SOXX -> Beta:  1.249713951357262
SOXX -> Tau:  0.00950129483097873



In [65]:
# run the SIMM optimization
expected_returns_list = list(expected_returns_dict.values())
beta_list = list(beta_dict.values())
tau_list = list(tau_dict.values())
alpha_list = list(alpha_dict.values())

simm_optimization = SingleFactorModelOptimization(expected_returns_list, beta_list, tau_list, market_index_data_variance, risk_free_rate)

In [70]:
# find the optimal weights when short selling is allowed
simm_weights_with_ss = simm_optimization.find_weights_short_selling_allowed()

In [72]:
simm_weights_with_ss_dict = {
  "VGT": simm_weights_with_ss[0],
  "XLK": simm_weights_with_ss[1],
  "SMH": simm_weights_with_ss[2],
  "IYT": simm_weights_with_ss[3],
  "SOXX": simm_weights_with_ss[4]
}

for ticker, weight in simm_weights_with_ss_dict.items():
    print(f"{ticker} -> SIMM Weights with Short Selling in %: ", weight * 100)

VGT -> SIMM Weights with Short Selling in %:  54.3398907819194
XLK -> SIMM Weights with Short Selling in %:  129.77111138520218
SMH -> SIMM Weights with Short Selling in %:  -48.840319357298426
IYT -> SIMM Weights with Short Selling in %:  -5.362673159201556
SOXX -> SIMM Weights with Short Selling in %:  -29.90800965062157


With weights found, we can find the expected return of portfolio and the risk using the formula:

$$
u_p = \alpha_p + \beta_p \cdot u_m

\newline

\sigma_p^2 = \beta_p^2 \cdot \sigma_m^2 + \tau_p^2

\newline

where:
\newline

\alpha_p = \sum_{i=1}^{n} x_i \cdot \alpha_i

\newline

\beta_p = \sum_{i=1}^{n} x_i \cdot \beta_i

\newline

\tau_p^2 = \sum_{i=1}^{n} x_i^2 \cdot \tau_i^2

In [68]:
simm_expected_portfolio_return_with_ss, simm_portfolio_risk_with_ss = Utils.calculate_factor_model_percentage_portfolio_return_and_risk(simm_weights_with_ss, market_index_data_expected_return, market_index_data_variance, alpha_list, beta_list, tau_list)
print("Expected Portfolio Return with Short Selling in %: ", simm_expected_portfolio_return_with_ss * 100)
print("Portfolio Risk with Short Selling in %: ", simm_portfolio_risk_with_ss * 100)

Expected Portfolio Return with Short Selling in %:  66.47693160638363
Portfolio Risk with Short Selling in %:  1.8471700984689663


In [69]:
# find the sharpe ratio
simm_sharpe_ratio_with_ss = Utils.calculate_sharpe_ratio(simm_expected_portfolio_return_with_ss, simm_portfolio_risk_with_ss, risk_free_rate)
print("SIMM Sharpe Ratio with Short Selling: ", simm_sharpe_ratio_with_ss)

SIMM Sharpe Ratio with Short Selling:  12.421633798767779


In [73]:
# find optimal weights when short selling is not allowed
simm_weights_no_ss = simm_optimization.find_weights_no_short_selling()

In [74]:
simm_weights_no_ss_dict = {
  "VGT": simm_weights_no_ss[0],
  "XLK": simm_weights_no_ss[1],
  "SMH": simm_weights_no_ss[2],
  "IYT": simm_weights_no_ss[3],
  "SOXX": simm_weights_no_ss[4]
}

for ticker, weight in simm_weights_no_ss_dict.items():
  print(f"{ticker} -> SIMM Weights without Short Selling in %: ", weight * 100)
    

VGT -> SIMM Weights without Short Selling in %:  25.002431213880545
XLK -> SIMM Weights without Short Selling in %:  53.45653136566564
SMH -> SIMM Weights without Short Selling in %:  2.162370545500286
IYT -> SIMM Weights without Short Selling in %:  17.85152152207638
SOXX -> SIMM Weights without Short Selling in %:  1.52714535287713


In [75]:
# we will now work to find the expected return and risk of the portfolio
simm_expected_portfolio_return_no_ss, simm_portfolio_risk_no_ss = Utils.calculate_factor_model_percentage_portfolio_return_and_risk(simm_weights_no_ss, market_index_data_expected_return, market_index_data_variance, alpha_list, beta_list, tau_list)
print("Expected Portfolio Return without Short Selling in %: ", simm_expected_portfolio_return_no_ss * 100)
print("Portfolio Risk without Short Selling in %: ", simm_portfolio_risk_no_ss * 100)

Expected Portfolio Return without Short Selling in %:  95.49843530512001
Portfolio Risk without Short Selling in %:  0.6865612006444066


In [76]:
# find the sharpe ratio
simm_sharpe_ratio_no_ss = Utils.calculate_sharpe_ratio(simm_expected_portfolio_return_no_ss, simm_portfolio_risk_no_ss, risk_free_rate)
print("SIMM Sharpe Ratio without Short Selling: ", simm_sharpe_ratio_no_ss)

SIMM Sharpe Ratio without Short Selling:  18.22253946823623
